In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = True

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFPSPJJ%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFPSPJJ%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_temperature_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempS = InterfaceData(size=inodes, value=T_bot)
TempF = InterfaceData(size=inodes, value=300.0)
TempS = InterfaceData(size=inodes, value=T_bot)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = RelativeCovergenceMonitor(tol=1e-5)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
hF2S = hF[f2s]
hS = solverS.get_interface_h(tag=interface_tag)
hS2F = hS[s2f]

CF1 = hS2F/(hS2F+hF)
CF2 = hF/(hS2F+hF)

CS1 = hS/(hF2S+hS)
CS2 = hF2S/(hF2S+hS)

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.01
Nmax = 8000
t = 0.0
avg_pc_iterations = 0.0
flog = open('FPSPJ%i.log' % k, mode='w')
flog.write('Fluid pseudo Robin with solid pseudo Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FPSPJ%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        TempF.backup()
        TempS.backup()
        
        # advance solvers
        solverF.advance(t=t, dt=dt)
        solverS.advance(t=t, dt=dt)
        
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # retrieve solid ambient temperature
        abTempS.curr[:] = solverS.get_interface_ambient_temperature(
            tag=interface_tag,
            h_given = hS
        )
        
        # update solid interface temperature
        TempS.curr[:] = CS1 * abTempS.curr + CS2 * abTempF.curr[f2s]
        # update fluid interface temperature
        TempF.curr[:] = CF1 * abTempS.curr[s2f] + CF2 * abTempF.curr
        
        # update residual
        TempF.update_res()
        TempS.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(TempF)
        if is_conv or pc_counts >= max_pc_steps:
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=TempF.curr
            )
            solverS.update_temperature_interface(
                tag=interface_tag,
                T=TempS.curr
            )
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(TempF)
                under_relax.update_solution(TempF)
#                 under_relax.determine_omega(TempS)
#                 under_relax.update_solution(TempS)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=TempF.curr
            )
            solverS.update_temperature_interface(
                tag=interface_tag,
                T=TempS.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-300)/10
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=200.
step=1, pc_iterations=83.
step=2, pc_iterations=47.
step=3, pc_iterations=55.
step=4, pc_iterations=13.
step=5, pc_iterations=21.
step=6, pc_iterations=37.
step=7, pc_iterations=38.
step=8, pc_iterations=23.
step=9, pc_iterations=48.
step=10, pc_iterations=6.
step=11, pc_iterations=25.
step=12, pc_iterations=12.
step=13, pc_iterations=44.
step=14, pc_iterations=17.
step=15, pc_iterations=32.
step=16, pc_iterations=7.
step=17, pc_iterations=6.
step=18, pc_iterations=13.
step=19, pc_iterations=16.
step=20, pc_iterations=20.
step=21, pc_iterations=14.
step=22, pc_iterations=11.
step=23, pc_iterations=11.
step=24, pc_iterations=6.
step=25, pc_iterations=9.
step=26, pc_iterations=17.
step=27, pc_iterations=16.
step=28, pc_iterations=7.
step=29, pc_iterations=12.
step=30, pc_iterations=9.
step=31, pc_iterations=19.
step=32, pc_iterations=11.
step=33, pc_iterations=14.
step=34, pc_iterations=8.
step=35, pc_iterations=15.
step=36, pc_iterations=6.
step=37, pc_iterati

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFPSPJ%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.91150429 0.91716801 0.92221157 0.92510822 0.92759653 0.92988256
 0.93189556 0.93371119 0.93545843 0.93700111 0.93843353 0.93975391
 0.94102506 0.94219567 0.94330134 0.94432403 0.94530987 0.94623265
 0.94709461 0.9479236  0.94869759 0.94943539 0.95013473 0.95078867
 0.95141723 0.95200767 0.95257564 0.95310015 0.95359209 0.95406329
 0.95451066 0.95492885 0.95532688 0.95570137 0.95605494 0.9563832
 0.95669721 0.95698931 0.95726972 0.95752662 0.95776577 0.95798728
 0.95819553 0.95839006 0.9585705  0.95873643 0.95889076 0.95903022
 0.95915566 0.95927333 0.95937696 0.95946982 0.95955058 0.9596208
 0.95967925 0.95972877 0.95976601 0.95979274 0.9598099  0.95981541
 0.95980837 0.95979414 0.95976709 0.95973064 0.9596805  0.95962095
 0.95954736 0.95946313 0.95936399 0.95925294 0.95913103 0.95899226
 0.95883356 0.9586651  0.95847694 0.95827467 0.95804948 0.95780639
 0.95753976 0.95725337 0.9569392  0.95659918 0.95623272 0.95583707
 0.95540591 0.95492968 0.95443042 0.95387541 0.95326141 0.952604